In [42]:
import pandas as pd
import numpy as np

In [43]:
# Attributes Information:

data_info = pd.read_csv("dataset/NUSW-NB15_features.csv", encoding = "ISO-8859-1")
data_info

,No.,Name,Type,Description
0,1,srcip,nominal,Source IP address
1,2,sport,integer,Source port number
2,3,dstip,nominal,Destination IP address
3,4,dsport,integer,Destination port number
4,5,proto,nominal,Transaction protocol
5,6,state,nominal,Indicates to the state and its dependent proto...
6,7,dur,Float,Record total duration
7,8,sbytes,Integer,Source to destination transaction bytes
8,9,dbytes,Integer,Destination to source transaction bytes
9,10,sttl,Integer,Source to destination time to live value


In [44]:
# Retrive Attribute Names

column_attributes = list (data_info['Name']) 
column_attributes

['srcip',
 'sport',
 'dstip',
 'dsport',
 'proto',
 'state',
 'dur',
 'sbytes',
 'dbytes',
 'sttl',
 'dttl',
 'sloss',
 'dloss',
 'service',
 'Sload',
 'Dload',
 'Spkts',
 'Dpkts',
 'swin',
 'dwin',
 'stcpb',
 'dtcpb',
 'smeansz',
 'dmeansz',
 'trans_depth',
 'res_bdy_len',
 'Sjit',
 'Djit',
 'Stime',
 'Ltime',
 'Sintpkt',
 'Dintpkt',
 'tcprtt',
 'synack',
 'ackdat',
 'is_sm_ips_ports',
 'ct_state_ttl',
 'ct_flw_http_mthd',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_srv_src',
 'ct_srv_dst',
 'ct_dst_ltm',
 'ct_src_ ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'attack_cat',
 'Label']

In [45]:
pd.set_option('display.max_column', 50)

In [60]:
# Read the data available in 4 csv files using temporary data frames

dataframe1 = pd.read_csv("dataset/UNSW-NB15_1.csv",low_memory=False, header=None)
dataframe2 = pd.read_csv("dataset/UNSW-NB15_2.csv", low_memory=False, header=None)
dataframe3 = pd.read_csv("dataset/UNSW-NB15_3.csv", low_memory=False, header=None)
dataframe4 = pd.read_csv("dataset/UNSW-NB15_4.csv",low_memory=False, header=None)

In [61]:
# Sample data

dataframe1[0:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0


In [48]:
# Name the columns of all data frame with attribute names in features.csv

df_list = [dataframe1,dataframe2,dataframe3,dataframe4]
for df in df_list:
    name = [v.lower() for k, v in enumerate(column_attributes)]
    df.columns = name

In [49]:
# sample result rows after renaming columns

dataframe1[0:5]

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,sload,dload,spkts,dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,sjit,djit,stime,ltime,sintpkt,dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0


In [50]:
# Concat the 4 temporary data frames into one single data frame

dataset = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4])
dataset.shape

(2540047, 49)

In [51]:
# Delete temporary data frames

del dataframe1
del dataframe2
del dataframe3
del dataframe4

In [52]:
# Get columns that have null values

dataset.columns[dataset.isna().any()].tolist()

['ct_flw_http_mthd', 'is_ftp_login', 'attack_cat']

In [53]:
# Handling Null values 

dataset.loc[:,'attack_cat'] = dataset.loc[:,'attack_cat'].replace(np.nan,'normal', regex=True)
dataset.loc[:,'ct_flw_http_mthd'] = dataset.loc[:,'ct_flw_http_mthd'].replace(np.nan,0, regex=True)
dataset.loc[:,'is_ftp_login'] = dataset.loc[:,'is_ftp_login'].replace(np.nan,0, regex=True)

In [54]:
# Drop unwanted columns from the dataset

dataset.drop(['srcip',
 'sport',
 'dstip',
 'dsport',
 'stime',
 'ltime'], axis = 1) 

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,sload,dload,spkts,dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,sjit,djit,sintpkt,dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,udp,CON,0.001055,132,164,31,29,0,0,dns,5.004739e+05,6.218009e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,3,7,1,3,1,1,1,normal,0
1,udp,CON,0.036133,528,304,31,29,0,0,-,8.767609e+04,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,normal,0
2,udp,CON,0.001119,146,178,31,29,0,0,dns,5.218945e+05,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,normal,0
3,udp,CON,0.001209,132,164,31,29,0,0,dns,4.367246e+05,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,normal,0
4,udp,CON,0.001169,146,178,31,29,0,0,dns,4.995722e+05,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,normal,0
5,udp,CON,0.078339,568,312,31,29,0,0,-,4.350323e+04,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,normal,0
6,udp,CON,0.001134,132,164,31,29,0,0,dns,4.656085e+05,5.784832e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,7,1,2,2,1,1,normal,0
7,arp,INT,0.000000,46,0,0,0,0,0,-,0.000000e+00,0.000000e+00,1,0,0,0,0,0,46,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,2,0.0,0.0,0,2,2,2,2,2,2,2,normal,0
8,udp,CON,0.001126,146,178,31,29,0,0,dns,5.186501e+05,6.323268e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,0.018000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,7,3,1,1,1,1,normal,0
9,udp,CON,0.001167,132,164,31,29,0,0,dns,4.524422e+05,5.621251e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,0.018000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,7,2,1,1,1,1,normal,0


In [55]:
X_train = dataset.drop(['attack_cat', 'label'], axis = 1)
num_keys = X_train.select_dtypes(exclude=['object']).keys()

In [56]:
def log_transform(num_train_dataset):
    # print(num_train_dataset)
    ds = num_train_dataset.apply(lambda x: np.log(x + 1))
    return ds

In [57]:
from sklearn.preprocessing import MinMaxScaler
def min_max_scaler(transformed_dataset):
    ds = MinMaxScaler().fit_transform(transformed_dataset)
    return ds

In [58]:
log_transform_ds = []
log_transform_ds = log_transform(X_train[num_keys])
print(log_transform_ds)
min_max_scale = min_max_scaler(log_transform_ds)
print(min_max_scale)

                 dur     sbytes     dbytes      sttl      dttl     sloss  \
0       1.054444e-03   4.890349   5.105945  3.465736  3.401197  0.000000   
1       3.549551e-02   6.270988   5.720312  3.465736  3.401197  0.000000   
2       1.118374e-03   4.990433   5.187386  3.465736  3.401197  0.000000   
3       1.208270e-03   4.890349   5.105945  3.465736  3.401197  0.000000   
4       1.168317e-03   4.990433   5.187386  3.465736  3.401197  0.000000   
5       7.542189e-02   6.343880   5.746203  3.465736  3.401197  0.000000   
6       1.133358e-03   4.890349   5.105945  3.465736  3.401197  0.000000   
7       0.000000e+00   3.850148   0.000000  0.000000  0.000000  0.000000   
8       1.125367e-03   4.990433   5.187386  3.465736  3.401197  0.000000   
9       1.166320e-03   4.890349   5.105945  3.465736  3.401197  0.000000   
10      0.000000e+00   3.850148   0.000000  0.000000  0.000000  0.000000   
11      0.000000e+00   3.850148   0.000000  0.000000  0.000000  0.000000   
12      0.00

[[1.16114098e-04 2.96750557e-01 3.09442522e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.90872353e-03 3.80528927e-01 3.46675795e-01 ... 0.00000000e+00
  0.00000000e+00 1.14981184e-01]
 [1.23154049e-04 3.02823706e-01 3.14378161e-01 ... 1.14981184e-01
  0.00000000e+00 0.00000000e+00]
 ...
 [2.19431362e-01 4.54936930e-01 4.63286387e-01 ... 1.14981184e-01
  1.18635908e-01 1.14981184e-01]
 [1.28116912e-01 4.95169869e-01 7.28468829e-01 ... 1.14981184e-01
  1.18635908e-01 1.14981184e-01]
 [7.31436552e-02 3.85588595e-01 3.94999251e-01 ... 1.14981184e-01
  1.18635908e-01 1.14981184e-01]]
